## This Notebook integrates all the wild data sources.
### This includes our data from a variety of sources, as well as data from Salmon Coast Field Station, Cedar Creek Field Station, and the Hakai Institute

In [72]:
import pandas as pd
from pathlib import Path

## Configuration variables for this notebook

In [73]:

# paths to files for our data
wild_data_dir = Path('.')
events_filepath = wild_data_dir / 'wild_sample_events.csv'
fish_lice_filepath = wild_data_dir / 'wild_fish_lice.csv'

# paths to the Salmon Coast Field Station data files
scfs_events_filepath = wild_data_dir / 'salmon_coast_wild_sample_events.csv'
scfs_fish_lice_filepath = wild_data_dir / 'salmon_coast_wild_fish_lice.csv'

# paths to the Cedar Creek Field Station data files
ccfs_events_filepath = wild_data_dir / 'cedar_coast_wild_sample_events.csv'
ccfs_fish_lice_filepath = wild_data_dir / 'cedar_coast_wild_fish_lice.csv'

# paths to the Hakai Institute data files
hakai_events_filepath = wild_data_dir / 'hakai_wild_sample_events.csv'
hakai_fish_lice_filepath = wild_data_dir / 'hakai_wild_fish_lice.csv'


# -- OUTPUT paths for writing the combined data

all_events_filepath = wild_data_dir / 'all_wild_sample_events.csv'
all_fish_lice_filepath = wild_data_dir / 'all_wild_fish_lice.csv'

## Join all the event data

In [74]:
# load the event data files
events_df = pd.read_csv(events_filepath, parse_dates=['sampledate'])
scfs_events_df = pd.read_csv(scfs_events_filepath, parse_dates=['sampledate'])
ccfs_events_df = pd.read_csv(ccfs_events_filepath, parse_dates=['sampledate'])
hakai_events_df = pd.read_csv(hakai_events_filepath, parse_dates=['sampledate'])

In [75]:
events_df.head()

,event_id,sampledate,region,dfozone,sample_site,latitude,longitude,source
0,1,2003-05-13,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.67950,Fisheries and Oceans Canada
1,2,2003-05-21,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.67950,Fisheries and Oceans Canada
2,3,2003-05-26,Broughton Archipelago,3_3,Adeane Point,50.71970,-125.67950,Fisheries and Oceans Canada
3,4,2003-06-02,Broughton Archipelago,3_3,Adeane Point,50.71970,-125.67950,Fisheries and Oceans Canada
4,5,2003-05-13,Broughton Archipelago,3_3,Adeane Point,50.73840,-125.67985,Fisheries and Oceans Canada


In [76]:
# prepend a string to the main event IDs. Currently they are numbers and the mixed types can confuse data parsing later
events_df['event_id'] = events_df.event_id.apply(lambda eid: 'bc_' + str(eid).strip())
events_df.head()

,event_id,sampledate,region,dfozone,sample_site,latitude,longitude,source
0,bc_1,2003-05-13,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.67950,Fisheries and Oceans Canada
1,bc_2,2003-05-21,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.67950,Fisheries and Oceans Canada
2,bc_3,2003-05-26,Broughton Archipelago,3_3,Adeane Point,50.71970,-125.67950,Fisheries and Oceans Canada
3,bc_4,2003-06-02,Broughton Archipelago,3_3,Adeane Point,50.71970,-125.67950,Fisheries and Oceans Canada
4,bc_5,2003-05-13,Broughton Archipelago,3_3,Adeane Point,50.73840,-125.67985,Fisheries and Oceans Canada


In [77]:
# concatenate all the files
all_events_df = pd.concat([events_df, scfs_events_df, ccfs_events_df, hakai_events_df], ignore_index=True, sort=False)
all_events_df.source.unique()

array(['Fisheries and Oceans Canada', 'Mainstream Biological Consulting',
       'Broughton Archipelago Monitoring Plan',
       'Marine Environmental Research Program', 'Marty Krkosek',
       'Pacificus Biological Services', 'Kitasoo First Nation',
       'Salmon Coast Field Station', 'Cedar Coast Field Station',
       'Hakai Institute'], dtype=object)

In [78]:
# make a useful source_code column - redundant but easier typing
source_code_mapping = {
    'Fisheries and Oceans Canada': 'DFO',
    'Mainstream Biological Consulting': 'MBC',
    'Broughton Archipelago Monitoring Plan': 'BAMP',
    'Marine Environmental Research Program': 'MERP',
    'Marty Krkosek': 'MK',
    'Pacificus Biological Services': 'Pacif',
    'Kitasoo First Nation': 'Kit',
    'Salmon Coast Field Station': 'SCS',
    'Cedar Coast Field Station': 'CC',
    'Hakai Institute': 'Hak'
}
all_events_df['source_code'] = all_events_df['source'].apply(
    lambda x: source_code_mapping[x] if x in source_code_mapping else None)
all_events_df.source_code.unique()

array(['DFO', 'MBC', 'BAMP', 'MERP', 'MK', 'Pacif', 'Kit', 'SCS', 'CC',
       'Hak'], dtype=object)

In [79]:
# document the protocol for fish sampling at each event
# (i) enter "Lethal" into all rows for 'protocol';
all_events_df['protocol'] = 'Lethal'
# (ii) over-write "Mixed" into all rows that have source_code = "Hak";
all_events_df.loc[all_events_df.source_code == 'Hak', 'protocol'] = 'Mixed'
# (iii) over-write "Non-lethal" into all rows that have source_code (IN) "SCS", "MK" or "CC"
all_events_df.loc[all_events_df.source_code.isin(['SCS', 'MK', 'CC']), 'protocol'] = 'Non-lethal'
# (iv) over-write back to "Lethal" all row that have [source_code = "SCS" AND sampledate < 01/01/2005]
#         (Salmon Coast only switched to non-lethal from 2005...)
start_2005 = pd.Timestamp(year=2005, month=1, day=1)
all_events_df.loc[(all_events_df.source_code == 'SCS') & (all_events_df['sampledate'] < start_2005), 'protocol'] = 'Lethal'

In [80]:
all_events_df[all_events_df.source_code == 'Hak'].head()

,event_id,sampledate,region,dfozone,sample_site,latitude,longitude,source,source_code,protocol
16110,hakai_DE100N2,2015-05-12,BA/DI border,NaN,Hurtado Point,NaN,NaN,Hakai Institute,Hak,Mixed
16111,hakai_DE100N1,2015-05-12,BA/DI border,NaN,Hurtado Point,NaN,NaN,Hakai Institute,Hak,Mixed
16112,hakai_DE100N3,2015-05-12,Discovery Islands,3_2,Hurtado Point,49.9613,-124.7470,Hakai Institute,Hak,Mixed
16113,hakai_DE101N1,2015-05-13,Discovery Islands,3_2,Kinghorn Island,50.07902,-124.8509,Hakai Institute,Hak,Mixed
16114,hakai_DE102N1,2015-05-14,Discovery Islands,3_2,Francisco Point,49.99932,-125.1395,Hakai Institute,Hak,Mixed


In [81]:
# write out to CSV
all_events_df.to_csv(all_events_filepath, index=False)

## Join all the fish/lice data

In [82]:
# load the fish/lice files
fish_lice_df = pd.read_csv(fish_lice_filepath)
scfs_fish_lice_df = pd.read_csv(scfs_fish_lice_filepath)
ccfs_fish_lice_df = pd.read_csv(ccfs_fish_lice_filepath)
hakai_fish_lice_df = pd.read_csv(hakai_fish_lice_filepath)

In [83]:
# prepend a string to the main event IDs to match events_df
# Currently they are numbers and the mixed types can confuse data parsing later
fish_lice_df['event_id'] = fish_lice_df.event_id.apply(lambda eid: 'bc_' + str(eid).strip())
fish_lice_df.head()

,event_id,fish_id,length,weight,height,fish_species,lep_cop,lep_chal,lep_motile,lep_unknown,cal_cop,cal_chal,cal_motile,cal_unknown,unknown_cop,unknown_chal,unknown_motile,unknown_unknown
0,bc_5666,1,44.0,0.96,NaN,Chum Salmon,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bc_5666,2,39.0,0.61,NaN,Pink Salmon,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bc_5666,3,45.0,0.94,NaN,Pink Salmon,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bc_5666,4,43.0,0.84,NaN,Chum Salmon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bc_5666,5,38.0,0.53,NaN,Pink Salmon,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# concatenate all the files
all_fish_lice_df = pd.concat([fish_lice_df, scfs_fish_lice_df, ccfs_fish_lice_df, hakai_fish_lice_df],
                             ignore_index=True, sort=False)

In [85]:
# remove the lice_protocol field as the information is now in events to make it easier for data management later
all_fish_lice_df.drop('lice_protocol', axis=1, inplace=True)
all_fish_lice_df.head()

,event_id,fish_id,length,weight,height,fish_species,lep_cop,lep_chal,lep_motile,lep_unknown,cal_cop,cal_chal,cal_motile,cal_unknown,unknown_cop,unknown_chal,unknown_motile,unknown_unknown
0,bc_5666,1,44.0,0.96,NaN,Chum Salmon,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bc_5666,2,39.0,0.61,NaN,Pink Salmon,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bc_5666,3,45.0,0.94,NaN,Pink Salmon,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bc_5666,4,43.0,0.84,NaN,Chum Salmon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bc_5666,5,38.0,0.53,NaN,Pink Salmon,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# write out to csv
all_fish_lice_df.to_csv(all_fish_lice_filepath, index=False)

In [87]:
list(all_fish_lice_df.fish_species.unique())

['Chum Salmon',
 'Pink Salmon',
 'Coho Salmon',
 'Three-Spined Stickleback',
 'Chinook Salmon',
 'Other Species',
 'Pacific Herring',
 'Sockeye Salmon']